# ML HW2 Decision Trees
### Fatma Ridaoui 1950044013

In [103]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
dataDf = pd.read_csv('abalone.data',sep=",")

In [3]:
col_names =["Sex","Length","Diameter","Height","Whole weight","Shucked weight","Viscera weight","Shell weight","Rings"]

In [4]:
dataDf.head()

,M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15,15
0,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
1,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
2,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
3,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
4,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8


In [5]:
dataDf.columns = col_names

In [6]:
dataDf.head()

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
1,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
2,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
3,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
4,I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8


In [7]:
data = np.array(dataDf)

In [25]:
def findUnique(rows,col):
    #find classes in dataset from label col
    return set([row[col] for row in rows])

In [26]:
#findUnique(data,-1) #CLASS 1-29

In [51]:
def attribute_find(data):
    attribute_types = []

    for each in data:
        if isinstance(each, int) or isinstance(each, float):
            attribute_types.append(1)
        else:
            attribute_types.append(2) #categorical
    return attribute_types

In [28]:
X = dataDf.drop("Rings",1)
y = dataDf["Rings"]
X_train, X_test,y_train,y_test=train_test_split(X,y,test_size= 0.2)

In [29]:
"""We will need for impurity
Counts the number of each type of example in a dataset."""
def class_counts(rows):
    counts = {} 
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [30]:
class_counts(data)

{7: 391,
 9: 689,
 10: 634,
 8: 568,
 20: 26,
 16: 67,
 19: 32,
 14: 126,
 11: 487,
 12: 267,
 15: 102,
 18: 42,
 13: 203,
 5: 115,
 4: 57,
 6: 259,
 21: 14,
 17: 58,
 22: 6,
 1: 1,
 3: 15,
 26: 1,
 23: 9,
 29: 1,
 2: 1,
 27: 2,
 25: 1,
 24: 2}

In [31]:
attribute_types =attribute_find(data)

In [66]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            col_names[self.column], condition, str(self.value))

In [67]:

def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

## Decision Tree


In [33]:
class Leaf:
    def __init__(self,rows):
        self.pred = class_counts(rows)

In [34]:
class DecisionNode:
    def __init__(self, quest,true,false):
        self.question = quest
        self.true = true
        self.false = false

In [106]:
def info_gain(left, right, current_uncertainty):

    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [107]:
def gini(rows):

    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [105]:
def partition(rows, question):

    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [42]:
def find_best_modeling_dt(rows):
 
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [57]:
def build_dt(X,y,attribute_types, options):
    # returns a decision tree
    gain, question = find_best_modeling_dt(X)
    if gain ==0:
        return Leaf(X)
    
    true_rows, false_rows = partition(X, question)
    
    true = build_dt(true_rows,y,attribute_types,options)
    false = build_dt(false_rows, y, attribute_types,options)
    
    return DecisionNode(question, true, false)

In [71]:
X_train = np.array(X_train)
dt = build_dt(X_train,y_train,attribute_types,0)

### Predict

In [85]:
def predict_dt(row, node):
    
    if isinstance(node, Leaf):
        return node.pred

    if node.question.match(row):
        return predict_dt(row, node.true)
    else:
        return predict_dt(row, node.false)

In [100]:
X_test = np.array(X_test)
predict_dt(X_test[0], dt)

{0.35: 1}

In [87]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [96]:
print_leaf(predict_dt(X_test[0], dt))

{0.35: '100%'}

In [108]:
for idx, row in enumerate(X_test): 
    print ("Actual: %s. Predicted: %s" %
           (y[idx], print_leaf(predict_dt(row, dt))))
#Predicted will be changed, mistake printed

Actual: 7. Predicted: {0.35: '100%'}
Actual: 9. Predicted: {0.0215: '100%'}
Actual: 10. Predicted: {0.255: '100%'}
Actual: 7. Predicted: {0.14300000000000002: '100%'}
Actual: 8. Predicted: {0.425: '100%'}
Actual: 20. Predicted: {0.185: '100%'}
Actual: 16. Predicted: {0.053: '100%'}
Actual: 9. Predicted: {0.26: '100%'}
Actual: 19. Predicted: {0.1: '100%'}
Actual: 14. Predicted: {0.1845: '100%'}
Actual: 10. Predicted: {0.11: '100%'}
Actual: 11. Predicted: {0.14300000000000002: '100%'}
Actual: 10. Predicted: {0.275: '100%'}
Actual: 10. Predicted: {0.215: '100%'}
Actual: 12. Predicted: {0.053: '100%'}
Actual: 7. Predicted: {0.4395: '100%'}
Actual: 10. Predicted: {0.1945: '100%'}
Actual: 7. Predicted: {0.115: '100%'}
Actual: 9. Predicted: {0.469: '100%'}
Actual: 11. Predicted: {0.195: '100%'}
Actual: 10. Predicted: {0.22: '100%'}
Actual: 12. Predicted: {0.235: '100%'}
Actual: 9. Predicted: {0.3: '100%'}
Actual: 10. Predicted: {0.09: '100%'}
Actual: 11. Predicted: {0.161: '100%'}
Actual: 11.